In [1]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName('Subquery').getOrCreate()

24/12/11 15:41:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
trips_df =  spark.read.csv('data/fhvhv_tripdata_2020-03.csv', inferSchema=True, header=True)

In [3]:
trips_df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [4]:
trips_df.createOrReplaceTempView('mobility_data')

In [5]:
query = '''
SELECT *
FROM mobility_data
LIMIT 5
'''

spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   null|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   null|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   null|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

In [6]:
query1 = '''
SELECT split(pickup_datetime, ' ')[0] AS pickup_date,
       count(*) AS trips
FROM mobility_data
GROUP BY pickup_date
'''

spark.sql(query1).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-16|391518|
| 2020-03-03|697880|
| 2020-03-06|872012|
| 2020-03-26|141607|
| 2020-03-05|731165|
| 2020-03-02|648986|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
| 2020-03-04|707879|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-13|660914|
| 2020-03-27|159339|
| 2020-03-22|162165|
| 2020-03-28|138456|
| 2020-03-01|784246|
| 2020-03-19|252773|
| 2020-03-09|628940|
+-----------+------+
only showing top 20 rows



In [7]:
spark.sql(query1).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#127, 'count(1) AS trips#128]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(pickup_datetime#18,  , -1)[0]], [split(pickup_datetime#18,  , -1)[0] AS pickup_date#127, count(1) AS trips#128L]
+- SubqueryAlias mobility_data
   +- Relation[hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocationID#21,SR_Flag#22] csv

== Optimized Logical Plan ==
Aggregate [split(pickup_datetime#18,  , -1)[0]], [split(pickup_datetime#18,  , -1)[0] AS pickup_date#127, count(1) AS trips#128L]
+- Project [pickup_datetime#18]
   +- Relation[hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocationID#21,SR_Flag#22] csv

== Physical Plan ==
*(2) HashAggregate(keys=[split(pickup_datetime#18,  , -1)[0]#132], functions=[count(1)], 

In [8]:
query2 = '''
SELECT pickup_date,
       count(*) AS trips
FROM
    (SELECT
        split(pickup_datetime, ' ')[0] AS pickup_date
        FROM mobility_data)
GROUP BY pickup_date
'''

spark.sql(query2).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-16|391518|
| 2020-03-03|697880|
| 2020-03-06|872012|
| 2020-03-26|141607|
| 2020-03-05|731165|
| 2020-03-02|648986|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
| 2020-03-04|707879|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-13|660914|
| 2020-03-27|159339|
| 2020-03-22|162165|
| 2020-03-28|138456|
| 2020-03-01|784246|
| 2020-03-19|252773|
| 2020-03-09|628940|
+-----------+------+
only showing top 20 rows



In [9]:
spark.sql(query2).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#153]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#152]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#152], [pickup_date#152, count(1) AS trips#153L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(pickup_datetime#18,  , -1)[0] AS pickup_date#152]
      +- SubqueryAlias mobility_data
         +- Relation[hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocationID#21,SR_Flag#22] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#152], [pickup_date#152, count(1) AS trips#153L]
+- Project [split(pickup_datetime#18,  , -1)[0] AS pickup_date#152]
   +- Relation[hvfhs_license_num#16,dispatching_base_num#17,pickup_datetime#18,dropoff_datetime#19,PULocationID#20,DOLocati

In [10]:
spark.stop()